In [1]:
from google.cloud import bigquery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
client = bigquery.Client.from_service_account_json(r"/Users/mi/Downloads/proprietary-start-1c533200ea95.json")

In [9]:
# with FARM_FINGERPRINT - hash function, returns exact same value each time on the same date
# to get 10% of the data use " = 8", or " = 9" instead of " > 8"
# but we cannot longer make date-specific predictions (e.g. for Christmas), as our dates are hashed now

splitting_query = """
SELECT 
    date,
    airline,
    departure_airport,
    departure_schedule,
    arrival_airport,
    arrival_delay
FROM `bigquery-samples.airline_ontime_data.flights`
WHERE MOD(ABS(FARM_FINGERPRINT(date)), 10) < 8
"""

In [16]:
def run_query(que):
    table = client.query(que).to_dataframe()
    return table

In [ ]:
df = run_query(splitting_query, client)
df.head()

# runs too long

<h1> Repeatable splitting </h1>

In this notebook explored the impact of different ways of creating machine learning datasets.

<p>

Repeatability is important in machine learning. If you do the same thing now and 5 minutes from now and get different answers, then it makes experimentation difficult. In other words, you will find it difficult to gauge whether a change you made has resulted in an improvement or not.

<h3> Create a simple machine learning model </h3>

The dataset's number of records is 70 million.
<p>
We want to predict the arrival delay of an airline based on the departure delay. The model that we will use is a zero-bias linear model:
$$ delay_{arrival} = \alpha * delay_{departure} $$
<p>
To train the model is to estimate a good value for $\alpha$. 
<p>
One approach to estimate alpha is to use this formula:
$$ \alpha = \frac{\sum delay_{departure} delay_{arrival} }{  \sum delay_{departure}^2 } $$
Because we'd like to capture the idea that this relationship is different for flights from New York to Los Angeles vs. flights from Austin to Indianapolis (shorter flight, less busy airports), we'd compute a different $alpha$ for each airport-pair.  For simplicity, model made only for flights between Denver and Los Angeles.

<h3> Naive Random Split </h3> (Not Repeatable)

In [11]:
compute_alpha = """
#standardSQL
SELECT 
    SAFE_DIVIDE(
    SUM(arrival_delay * departure_delay),
    SUM(departure_delay * departure_delay)) AS alpha
FROM
(
    SELECT
        RAND() AS splitfield,
        arrival_delay,
        departure_delay
    FROM
        `bigquery-samples.airline_ontime_data.flights`
    WHERE
        departure_airport = 'DEN'
        AND arrival_airport = 'LAX'
)
WHERE
    splitfield < 0.8
"""

In [ ]:
results = bigquery.Client().query(compute_alpha).to_dataframe()
alpha = results['alpha'][0]
print(alpha)

<h3> How do we correctly train and evaluate? </h3>
<br/>
Here's the right way to compute the RMSE using the actual training and held-out (evaluation) data. Note how much harder this feels.

Although the calculations are correct, the experiment is still not repeatable.

In [12]:
train_and_eval_rand = """
#standardSQL
WITH
    alldata AS (
        SELECT
            IF (RAND() < 0.8, 'train', 'eval') AS dataset,
            arrival_delay,
            departure_delay
        FROM
            `bigquery-samples.airline_ontime_data.flights`
        WHERE
        departure_airport = 'DEN'
        AND arrival_airport = 'LAX' ),
    training AS (
        SELECT
            SAFE_DIVIDE(
                SUM(arrival_delay * departure_delay),
                SUM(departure_delay * departure_delay)) AS alpha
        FROM
            alldata
        WHERE
            dataset = 'train' )

SELECT
    MAX(alpha) AS alpha,
    dataset,
    SQRT(
        AVG(
            (arrival_delay - alpha * departure_delay) *
            (arrival_delay - alpha * departure_delay)
        )
    ) AS rmse,
    COUNT(arrival_delay) AS num_flights
FROM
    alldata,
    training
GROUP BY
    dataset
"""

In [17]:
df = run_query(train_and_eval_rand)

In [18]:
df.head()

,alpha,dataset,rmse,num_flights
0,0.975158,train,13.002381,63831
1,0.975158,eval,13.406476,15858


<h3> Data split using HASH </h3>

In [21]:
compute_alpha2 = """
#standardSQL
SELECT 
    SAFE_DIVIDE(
        SUM(arrival_delay * departure_delay),
        SUM(departure_delay * departure_delay)) AS alpha
FROM
    `bigquery-samples.airline_ontime_data.flights`
WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX'
    AND ABS(MOD(FARM_FINGERPRINT(date), 10)) < 8
"""

In [22]:
df = run_query(compute_alpha2)
alpha = df['alpha'][0]
print(alpha)

0.9758039143620403


In [23]:
# We can now use the alpha to compute RMSE. Because the alpha value is repeatable, we don't need to worry 
# that the alpha in the compute_rmse will be different from the alpha computed in the compute_alpha.

compute_rmse = """
#standardSQL
SELECT
    IF(ABS(MOD(FARM_FINGERPRINT(date), 10)) < 8, 'train', 'eval') AS dataset,
    SQRT(
        AVG(
            (arrival_delay - ALPHA * departure_delay) *
            (arrival_delay - ALPHA * departure_delay)
        )
    ) AS rmse,
    COUNT(arrival_delay) AS num_flights
FROM
    `bigquery-samples.airline_ontime_data.flights`
WHERE
    departure_airport = 'DEN'
    AND arrival_airport = 'LAX'
GROUP BY
    dataset
"""

In [24]:
print(client.query(compute_rmse.replace('ALPHA', str(alpha))).to_dataframe().head())

  dataset       rmse  num_flights
0    eval  12.764685        15671
1   train  13.160712        64018
